In [ ]:
import pandas as pd
import os, fnmatch
from datetime import date
import warnings
import numpy as np

In [ ]:
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 100000
pd.options.display.max_columns = 1000000

# Criando Funcoes

In [ ]:
def import_base(path, name, sheet = 0):
    os.chdir(path)
    xlsx_files = fnmatch.filter(os.listdir(path), '*' + name + '*.xlsx')
    dfbc = [pd.read_excel(xlsx_file, sheet_name=sheet) for xlsx_file in xlsx_files]
    base = pd.concat(dfbc, sort=False)
    return base

# Importando Bases

In [ ]:
#Movimentacao KMM
base_mov = pd.read_excel('f1858c78-f285-490f-8b5f-46bea0b576e8.xlsx')

In [ ]:
base_mov['CLIENTE'] = [x.split(' ')[0] for x in base_mov['CLIENTE']]

In [ ]:
#Movimentacao SADE
base_sade = import_base('/home/victhor/Projetos_Python/Painel', 'SANTOS', 'BASE SADE')

# Separacao pelo TIPO MOVIMENTACAO

In [ ]:
base_entrada = base_mov.query('`TIPO MOVIMENTAÇÃO` == "1. (+) Entrada Rodoviaria"')

In [ ]:
base_saida = base_mov.query('`TIPO MOVIMENTAÇÃO` == "4. (-) Expedido Ferrovia"')

# Tratando base_saida - REFATORAR - Agrupar pelo registros igual a SADE

In [ ]:
base_saida['DATA MOVIMENTAÇÃO'] = pd.to_datetime(base_saida['DATA MOVIMENTAÇÃO'])

In [ ]:
base_saida['DATA CARREGAMENTO'] = pd.to_datetime(base_saida['DATA CARREGAMENTO'])

In [ ]:
base_saida_agrupado = base_saida.groupby([
                                pd.to_datetime(base_saida['DATA MOVIMENTAÇÃO'].dt.strftime("%Y-%m-%d")),
                                'DATA MOVIMENTAÇÃO',
                                'PLACA',
                                'CONTRATO',
                                'CLIENTE',
                                'ORIGEM FISCAL', 
                                'ORIGEM FÍSICA',
                                'PROPRIETÁRIO',
                                'DATA FATURAMENTO', 
                                'DATA CARREGAMENTO', 
                                'TERMINAL DESTINO',
                                'PRODUTO'
                                    ], as_index=False).agg({
                                    'NOTA FISCAL': ' / '.join,
                                    'PESO DECLARADO':'sum', 
                                    'PESO RECEBIDO':'sum',
                                    'PESO SELECIONADO':'sum'
                                })

In [ ]:
base_saida_agrupado['PLACA'] = base_saida_agrupado['PLACA'].str.replace(r'\D', '').replace('', float('NaN'))
base_saida_agrupado.dropna(subset=['PLACA'], inplace=True)
base_saida_agrupado['PLACA'] = base_saida_agrupado['PLACA'].astype('int64')

# Tratando base_sade

In [ ]:
base_sade['CARGA'] = pd.to_datetime(base_sade['CARGA'])
base_sade['PESO'] = base_sade['PESO'].round(3)

In [ ]:
base_sade['CARGA'] = pd.to_datetime(base_sade['CARGA'])

# MERGE base_sade x base_saida

In [ ]:
base_sade.sort_values(by='CARGA', inplace=True)
base_saida_agrupado.sort_values(by='DATA CARREGAMENTO', inplace=True)

In [ ]:
print(base_sade.CARGA.dtype)
print(base_saida_agrupado['DATA CARREGAMENTO'].dtype)

In [ ]:
base_saida_conciliada = pd.merge_asof(base_saida_agrupado, 
                                        base_sade[[
                                                'COD_VAGÃO', 
                                                'COD_EST_CARGA',
                                                'MERCADORIA',
                                                'CLIENTE_CORRENTISTA', 
                                                'CARGA', 
                                                'DESCARGA', 
                                                'PESO',
                                                'TERMINAL', 
                                                'PESO DESTINO',
                                                'CONCILIADO CLIENTE?'
                                                ]],  
                                    left_on = 'DATA CARREGAMENTO',
                                    right_on = 'CARGA',
                                    left_by = ['ORIGEM FISCAL', 'PLACA', 'PESO SELECIONADO'],
                                    right_by = ['COD_EST_CARGA', 'COD_VAGÃO', 'PESO'],
                                    allow_exact_matches = True,
                                    direction = 'nearest',
                                    tolerance = pd.Timedelta("10 days"))

In [ ]:
base_saida_conciliada[base_saida_conciliada['PESO DESTINO'] != type(float)]

In [ ]:
base